In [9]:
from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import glob, os

In [2]:
data = pd.read_csv("data/fixed_data/AA.csv", index_col=0)
data = data.sort_index(ascending=True)
data

,open,close
2021-03-08 09:16:00,29.450,29.45
2021-03-08 09:17:00,29.460,29.46
2021-03-08 09:18:00,29.470,29.47
2021-03-08 09:19:00,29.480,29.48
2021-03-08 09:20:00,29.490,29.49
...,...,...
2021-03-19 15:58:00,31.275,31.24
2021-03-19 15:59:00,31.235,31.11
2021-03-19 16:00:00,31.115,31.14
2021-03-19 16:01:00,31.140,31.14


In [3]:
def plot_prices(open_y, close_y=None, x=None, buy_x=None, sale_x=None, title=None, save_to=None):
    fig = go.Figure()

    # basic plot for opening prices
    fig.add_trace(go.Scatter(y=open_y, x=x, mode='lines', name='Open', marker_color='rgba(255, 69, 0, .9)'))
    
    # add plot for closing prices
    if close_y is not None:
        fig.add_trace(go.Scatter(y=close_y, x=x, mode='lines', name='Close', marker_color='rgba(0, 0, 200, .9)'))
    
    # add points for buy
    if buy_x is not None:
        assert x is not None, "argument 'x' cant be None when using buy_x/sale_x"
        buy_y = pd.DataFrame(np.array(open_y), columns=["temp"], index=x).loc[buy_x, "temp"]
        fig.add_trace(go.Scatter(y=buy_y, x=buy_y.index, mode='markers', name='buy', marker_color='rgba(0, 200, 0, .9)', marker_size=10))
    
    # add points for selling
    if sale_x is not None:
        assert x is not None, "argument 'x' cant be None when using buy_x/sale_x"
        assert close_y is not None, "sale is calculated on close. Can't be None."
        sale_y = pd.DataFrame(np.array(close_y), columns=["temp"], index=x).loc[sale_x, "temp"]
        fig.add_trace(go.Scatter(y=sale_y, x=sale_y.index, mode='markers', name='sale', marker_color='rgba(200, 0, 0, .9)', marker_size=10))
    
    # add title
    if title is not None:
        fig.update_layout(title=title,
                       xaxis_title='Time -->',
                       yaxis_title='Price -->')
    
    # save+show figure
    if save_to is not None:
        fig.write_html(save_to)
    fig.show()

# Strategy 1

In [4]:
'''
Simplest(stupidest) strategy:
If price of stock is less than average of the last few stocks, make a immediate transaction.
'''

_data = data.loc[list(filter(lambda x: "2021-02-24" in x.__str__(), data.index))].sort_index(ascending=True)

last = 5
principal_start = 100
transaction_cost = 0.03/100

for last in range(1, 20):
    opening = _data['open']
    closing = _data['close']

    principal = principal_start
    transactions = 0

    for i in range(last, opening.shape[0]):
        average = np.mean(closing[i-last:i])

        if opening[i] < average:
            principal *= (1 - transaction_cost) # brokerage / TC
            principal *= (1 + (closing[i] - opening[i]) / opening[i]) # profit
        
        transactions += 1
    print(last, transactions, principal, (principal-principal_start)/principal_start*100)

1 0 100 0.0
2 0 100 0.0
3 0 100 0.0
4 0 100 0.0
5 0 100 0.0
6 0 100 0.0
7 0 100 0.0
8 0 100 0.0
9 0 100 0.0
10 0 100 0.0
11 0 100 0.0
12 0 100 0.0
13 0 100 0.0
14 0 100 0.0
15 0 100 0.0
16 0 100 0.0
17 0 100 0.0
18 0 100 0.0
19 0 100 0.0


# Strategy 2

In [26]:
'''
A little better strategy:
If price of stock is very close to the minimum of the last few prices, buy.
If price of stock is very close to the maximum of the last few prices, sell.
'''
min_low = 0.1
min_high = 0.3

max_low = 0.7
max_high = 0.9

last = 25 # history to look at

principal_start = 100
transaction_cost = 0.085/100

def strategy2(data, plot=False):
    opening = data['open']
    closing = data['close']
    buy_list = [False]*last
    sale_list = [False]*last

    principal = principal_start
    transactions = 0
    bought = False
    last_bought_at = 0

    for i in range(last, opening.shape[0]):
        # get history
        history = np.array(opening[i-last:i])
        
        # remove outliers
        low = np.percentile(history, 1)
        high  = np.percentile(history, 99)
        history = np.clip(history, low, high)
        
        if not bought:
            cur_price = opening[i]
            miny = history.min()
            maxy = history.max()
            low = miny + (maxy - miny) * min_low
            high = miny + (maxy - miny) * min_high
            
            # buy if price in appropriate range
            if cur_price < high and cur_price > low:
                bought = True
                principal *= (1 - transaction_cost) # reduce brokerage / TC
                last_bought_at = cur_price

                # add entry in buy list
                buy_list.append(True)
            else:
                buy_list.append(False)
            sale_list.append(False)
        else:
            cur_price = closing[i]
            miny = history.min()
            maxy = history.max()
            low = miny + (maxy - miny) * max_low
            high = miny + (maxy - miny) * max_high
            
            # selff if price in appropriate range
            if (cur_price < high and cur_price > low):
                bought = False
                profit_perc = (1 + (cur_price - last_bought_at) / last_bought_at)
                principal *= (1 + (cur_price - last_bought_at) / last_bought_at) # profit
                sale_list.append(True)
            else:
                sale_list.append(False)
            
            buy_list.append(False)

    if plot:
        plot_prices(open_y=opening,
                   close_y=closing,
                   x=_data.index,
                   title=date + f' (#buy={sum(buy_list)}, #sale={sum(sale_list)})',
                   buy_x=buy_list,
                   sale_x=sale_list,
#                        save_to=date+'.html'
                   )
    return (last, bought, sum(buy_list), sum(sale_list), 
          principal, (principal-principal_start)/principal_start*100)

In [27]:
from functools import reduce

folder = "data/fixed_data"
ultra_profit_list = []
for file in glob.glob(os.path.join(folder, "*.csv")):
    df = pd.read_csv(file, index_col=0, parse_dates=True)
    dates = list(set(list(map(lambda x: x.date(), df.index))))
    dates.sort()
    profit_list = []
    
    for date in dates:
        _data = df.loc[list(filter(lambda x: str(date) in x.__str__(), df.index))].sort_index(ascending=True)
        result = strategy2(_data)
        profit_list.append(result[-1])
    
    profit_list = 1 + np.array(profit_list) / 100
    
    total_profit = reduce(lambda a, b: a*b, profit_list) * 100 - 100
    ultra_profit_list.append(total_profit)
    print(f"{file} || {total_profit}%")
#     if abs(total_profit) > 5:
#         print(profit_list)
#         print()

data/fixed_data/TPR.csv || -0.5070619423386091%
data/fixed_data/FTI.csv || -9.878107310945182%
data/fixed_data/BCS.csv || -2.1327204056198497%
data/fixed_data/C.csv || -5.0469668800643035%
data/fixed_data/HSBC.csv || -5.550227878176273%
data/fixed_data/MOS.csv || -3.023644478156598%
data/fixed_data/AM.csv || -10.728658244378039%
data/fixed_data/BMY.csv || -3.86746365496127%
data/fixed_data/AMWL.csv || -2.77267449940625%
data/fixed_data/SPCE.csv || 25.277198011415635%
data/fixed_data/BHC.csv || -1.5500572957719072%
data/fixed_data/UAA.csv || -3.1065070495815803%
data/fixed_data/TME.csv || 6.3181948395167495%
data/fixed_data/UNP.csv || -5.5935392192338895%
data/fixed_data/RIO.csv || -5.063050619328877%
data/fixed_data/AMT.csv || 0.40296120959651205%
data/fixed_data/AA.csv || 9.33829959261756%
data/fixed_data/BEKE.csv || 0.6711348374287525%
data/fixed_data/CNK.csv || 3.5520843483289894%
data/fixed_data/AZUL.csv || -4.018122826075313%
data/fixed_data/TDOC.csv || 2.519597222645757%
data/fix

In [29]:
sum(ultra_profit_list)

86.77551641409484